# Dynamic pricing

* A pricing strategy that adjusts the price of a product or service in real-time
* Based on factors:
  * Market demand
  * Competition
  * Customer behaviour
* Goal is to maximize profits 
  * By finding the optimal price point that customers are willing to pay
  * Costs associated with producing and selling the product


## Algorithms

### Rule-based
Uses pre-defined rules to set prices i.e. increase price if demand is high or lower price if demand is low.

### Time-series forecasting
Uses historical data to forecast future demand and adjust prices accordingly. Takes into account seasonal patterns and trends to optimize pricing.

### Machine Learning algorithm
Use historical data to learn patterns and adjust prices based on predictive models. Can include regression analysis, decision trees, and neural networks.

### Reinforcement Learning algorithm
Uses trial and error to optimize pricing. Algorithm makes decisions based on feedback from customer behaviour and ajusts prices to maximize profit.

### Multi-armed bandit algorithm
Used in situations where there are multiple products or pricing options. Uses trial and error to test different pricing strategies and learns which ones are most effective in maximizing profit.
